In [1]:
import string
import numpy as np
from PIL import Image
import os
from pickle import dump, load
import numpy as np

from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout

# small library for seeing the progress of loops.
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

Using TensorFlow backend.


C:\Users\Admin\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
# Loading a text file into memory
def load_doc(filename):
    # Opening the file as read only
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

In [3]:
# get all imgs with their captions
def all_img_captions(filename):
    file = load_doc(filename)
    captions = file.split('\n')
    descriptions ={}
    for caption in captions[:-1]:
        img, caption = caption.split('\t')
        if img[:-2] not in descriptions:
            descriptions[img[:-2]] = [caption]
        else:
            descriptions[img[:-2]].append(caption)
    return descriptions

In [4]:
##Data cleaning- lower casing, removing puntuations and words containing numbers
def cleaning_text(captions):
    table = str.maketrans('','',string.punctuation)
    for img,caps in captions.items():
        for i,img_caption in enumerate(caps):

            img_caption.replace("-"," ")
            desc = img_caption.split()

            #converts to lower case
            desc = [word.lower() for word in desc]
            #remove punctuation from each token
            desc = [word.translate(table) for word in desc]
            #remove hanging 's and a 
            desc = [word for word in desc if(len(word)>1)]
            #remove tokens with numbers in them
            desc = [word for word in desc if(word.isalpha())]
            #convert back to string

            img_caption = ' '.join(desc)
            captions[img][i]= img_caption
    return captions

In [5]:
def text_vocabulary(descriptions):
    # build vocabulary of all unique words
    vocab = set()
    
    for key in descriptions.keys():
        [vocab.update(d.split()) for d in descriptions[key]]
    
    return vocab

In [6]:
#All descriptions in one file 
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + '\t' + desc )
    data = "\n".join(lines)
    file = open(filename,"w")
    file.write(data)
    file.close()

In [7]:
# all_train_captions = []
# for key, val in descriptions.items():
#     for cap in val:
#         all_train_captions.append(cap)

# # Consider only words which occur at least 8 times in the corpus
# word_count_threshold = 8
# word_counts = {}
# nsents = 0
# for sent in all_train_captions:
#     nsents += 1
#     for w in sent.split(' '):
#         word_counts[w] = word_counts.get(w, 0) + 1

# vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]

# print('preprocessed words %d ' % len(vocab))


In [8]:
#link to the dataset text
dataset_text = "Flickr_8k_text"
#link to the dataset images
dataset_images = "Flickr8k_Dataset/Flicker8k_Dataset"

In [9]:
#we prepare our text data
filename = dataset_text + "/" + "Flickr8k.token.txt"
#loading the file that contains all data
#mapping them into descriptions dictionary img to 5 captions
descriptions = all_img_captions(filename)
print("Length of descriptions =" ,len(descriptions))

#cleaning the descriptions
clean_descriptions = cleaning_text(descriptions)

#building vocabulary 
vocabulary = text_vocabulary(clean_descriptions)
print("Length of vocabulary = ", len(vocabulary))

#saving each description to file 
save_descriptions(clean_descriptions, "descriptions.txt")

Length of descriptions = 8092
Length of vocabulary =  8763


In [10]:

def extract_features(directory):
        model = Xception( include_top=False, pooling='avg' )
        features = {}
        for img in tqdm(os.listdir(directory)):
            filename = directory + "/" + img
            image = Image.open(filename)
            image = image.resize((299,299))
            image = np.expand_dims(image, axis=0)
            #image = preprocess_input(image)
            image = image/127.5
            image = image - 1.0
            
            feature = model.predict(image)
            features[img] = feature
        return features

In [11]:
#2048 feature vector
features = extract_features(dataset_images)
dump(features, open("features.p","wb"))

In [12]:
features = load(open("features.p","rb"))


In [13]:
import graphviz
import pydot
#load the data 
def load_photos(filename):
    file = load_doc(filename)
    photos = file.split("\n")[:-1]
    return photos


def load_clean_descriptions(filename, photos):   
    #loading clean_descriptions
    file = load_doc(filename)
    descriptions = {}
    for line in file.split("\n"):
        
        words = line.split()
        if len(words)<1 :
            continue
    
        image, image_caption = words[0], words[1:]
        
        if image in photos:
            if image not in descriptions:
                descriptions[image] = []
            desc = '<start> ' + " ".join(image_caption) + ' <end>'
            descriptions[image].append(desc)

    return descriptions


def load_features(photos):
    #loading all features
    all_features = load(open("features.p","rb"))
    #selecting only needed features
    features = {k:all_features[k] for k in photos}
    return features


In [14]:
filename = dataset_text + "/" + "Flickr_8k.trainImages.txt"

#train = loading_data(filename)
train_imgs = load_photos(filename)
train_descriptions = load_clean_descriptions("descriptions.txt", train_imgs)
train_features = load_features(train_imgs)

In [15]:
#converting dictionary to clean list of descriptions
def dict_to_list(descriptions):
    all_desc = []
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

#creating tokenizer class 
#this will vectorise text corpus
#each integer will represent token in dictionary 

from keras.preprocessing.text import Tokenizer

def create_tokenizer(descriptions):
    desc_list = dict_to_list(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(desc_list)
    return tokenizer


In [16]:
# give each word a index, and store that into tokenizer.p pickle file
tokenizer = create_tokenizer(train_descriptions)
dump(tokenizer, open('tokenizer.p', 'wb'))
vocab_size = len(tokenizer.word_index) + 1
vocab_size 

7577

In [17]:
#calculate maximum length of descriptions
def max_length(descriptions):
    desc_list = dict_to_list(descriptions)
    return max(len(d.split()) for d in desc_list)

max_length = max_length(descriptions)
max_length

32

In [18]:
features['1000268201_693b08cb0e.jpg'][0]

array([0.3592277 , 0.03271922, 0.03686446, ..., 0.13820337, 0.02857174,
       0.30709386], dtype=float32)

In [19]:
# Define the model

#1 Photo feature extractor - we extracted features from pretrained model Xception. 
#2 Sequence processor - word embedding layer that handles text, followed by LSTM 
#3 Decoder - Both 1 and 2 model produce fixed length vector. They are merged together and processed by dense layer to make final prediction

In [20]:
#create input-output sequence pairs from the image description.

#data generator, used by model.fit_generator()
def data_generator(descriptions, features, tokenizer, max_length):
    while 1:
        for key, description_list in descriptions.items():
            #retrieve photo features
            feature = features[key][0]
            input_image, input_sequence, output_word = create_sequences(tokenizer, max_length, description_list, feature)
            yield [[input_image, input_sequence], output_word]         

def create_sequences(tokenizer, max_length, desc_list, feature):
    X1, X2, y = list(), list(), list()
    # walk through each description for the image
    for desc in desc_list:
        # encode the sequence
        seq = tokenizer.texts_to_sequences([desc])[0]
        # split one sequence into multiple X,y pairs
        for i in range(1, len(seq)):
            # split into input and output pair
            in_seq, out_seq = seq[:i], seq[i]
            # pad input sequence
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            # encode output sequence
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            # store
            X1.append(feature)
            X2.append(in_seq)
            y.append(out_seq)
    return np.array(X1), np.array(X2), np.array(y)

In [21]:
[a,b],c = next(data_generator(train_descriptions, features, tokenizer, max_length))
a.shape, b.shape, c.shape

((47, 2048), (47, 32), (47, 7577))

In [33]:
from keras.utils import plot_model

# define the captioning model
def define_model(vocab_size, max_length):
    
    # features from the CNN model squeezed from 2048 to 256 nodes
    inputs1 = Input(shape=(2048,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)

    # LSTM sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)

    # Merging both models
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    # summarize model
    print(model.summary())
    plot_model(model, to_file='model.png', show_shapes=True)
    
    return model

In [34]:
# train our model

print('Dataset: ', len(train_imgs))
print('Descriptions: train=', len(train_descriptions))
print('Photos: train=', len(train_features))
print('Vocabulary Size:', vocab_size)
print('Description Length: ', max_length)

model = define_model(vocab_size, max_length)
epochs = 10
steps = len(train_descriptions)
# making a directory models to save our models
os.mkdir("models")
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
    model.fit_generator(generator, epochs=1, steps_per_epoch= steps, verbose=1)
    model.save("models/model_" + str(i) + ".h5")

Dataset:  6000
Descriptions: train= 6000
Photos: train= 6000
Vocabulary Size: 7577
Description Length:  32
Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 32)           0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 32, 256)      1939712     input_17[0][0]                   
__________________________________________________________________________________________________
dropout_15 (Dropout)            (None, 2048)         0           input_16[0][0]     

C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1


 313/6000 [>.............................] - ETA: 2:14:47 - loss: 8.91 - ETA: 1:21:50 - loss: 8.91 - ETA: 1:04:27 - loss: 8.88 - ETA: 55:38 - loss: 8.8679 - ETA: 49:41 - loss: 8.83 - ETA: 45:29 - loss: 8.78 - ETA: 42:58 - loss: 8.70 - ETA: 41:28 - loss: 8.62 - ETA: 40:17 - loss: 8.49 - ETA: 38:34 - loss: 8.39 - ETA: 37:32 - loss: 8.26 - ETA: 36:54 - loss: 8.15 - ETA: 36:20 - loss: 8.28 - ETA: 35:33 - loss: 8.21 - ETA: 34:58 - loss: 8.14 - ETA: 34:28 - loss: 8.04 - ETA: 34:14 - loss: 8.05 - ETA: 33:36 - loss: 7.89 - ETA: 33:03 - loss: 7.82 - ETA: 32:47 - loss: 7.74 - ETA: 32:32 - loss: 7.70 - ETA: 32:12 - loss: 7.62 - ETA: 31:57 - loss: 7.57 - ETA: 31:53 - loss: 7.51 - ETA: 31:51 - loss: 7.46 - ETA: 31:47 - loss: 7.43 - ETA: 31:36 - loss: 7.36 - ETA: 31:23 - loss: 7.37 - ETA: 31:18 - loss: 7.35 - ETA: 31:18 - loss: 7.33 - ETA: 31:08 - loss: 7.28 - ETA: 31:03 - loss: 7.27 - ETA: 30:48 - loss: 7.27 - ETA: 30:40 - loss: 7.24 - ETA: 30:29 - loss: 7.26 - ETA: 30:21 - loss: 7.26 - ETA: 30:12 

2197/6000 [=========>....................] - ETA: 23:54 - loss: 4.99 - ETA: 23:54 - loss: 4.99 - ETA: 23:53 - loss: 4.99 - ETA: 23:53 - loss: 4.99 - ETA: 23:53 - loss: 4.99 - ETA: 23:52 - loss: 4.99 - ETA: 23:52 - loss: 4.99 - ETA: 23:52 - loss: 4.99 - ETA: 23:51 - loss: 4.99 - ETA: 23:51 - loss: 4.99 - ETA: 23:51 - loss: 4.99 - ETA: 23:50 - loss: 4.99 - ETA: 23:50 - loss: 4.99 - ETA: 23:50 - loss: 4.99 - ETA: 23:49 - loss: 4.99 - ETA: 23:49 - loss: 4.99 - ETA: 23:48 - loss: 4.99 - ETA: 23:48 - loss: 4.99 - ETA: 23:48 - loss: 4.99 - ETA: 23:47 - loss: 4.99 - ETA: 23:47 - loss: 4.99 - ETA: 23:47 - loss: 4.99 - ETA: 23:46 - loss: 4.99 - ETA: 23:46 - loss: 4.99 - ETA: 23:46 - loss: 4.99 - ETA: 23:45 - loss: 4.99 - ETA: 23:45 - loss: 4.99 - ETA: 23:45 - loss: 4.99 - ETA: 23:44 - loss: 4.99 - ETA: 23:44 - loss: 4.99 - ETA: 23:44 - loss: 4.99 - ETA: 23:43 - loss: 4.99 - ETA: 23:43 - loss: 4.99 - ETA: 23:42 - loss: 4.99 - ETA: 23:42 - loss: 4.99 - ETA: 23:42 - loss: 4.99 - ETA: 23:41 - loss: 

...

5338/6000 [=========================>....] - ETA: 3:15 - loss: 3.656 - ETA: 3:15 - loss: 3.656 - ETA: 3:15 - loss: 3.656 - ETA: 3:14 - loss: 3.656 - ETA: 3:14 - loss: 3.656 - ETA: 3:14 - loss: 3.656 - ETA: 3:14 - loss: 3.656 - ETA: 3:14 - loss: 3.656 - ETA: 3:13 - loss: 3.656 - ETA: 3:13 - loss: 3.656 - ETA: 3:13 - loss: 3.656 - ETA: 3:13 - loss: 3.656 - ETA: 3:13 - loss: 3.656 - ETA: 3:12 - loss: 3.657 - ETA: 3:12 - loss: 3.657 - ETA: 3:12 - loss: 3.657 - ETA: 3:12 - loss: 3.657 - ETA: 3:12 - loss: 3.657 - ETA: 3:11 - loss: 3.657 - ETA: 3:11 - loss: 3.657 - ETA: 3:11 - loss: 3.657 - ETA: 3:11 - loss: 3.657 - ETA: 3:11 - loss: 3.657 - ETA: 3:10 - loss: 3.657 - ETA: 3:10 - loss: 3.657 - ETA: 3:10 - loss: 3.657 - ETA: 3:10 - loss: 3.657 - ETA: 3:10 - loss: 3.657 - ETA: 3:09 - loss: 3.657 - ETA: 3:09 - loss: 3.657 - ETA: 3:09 - loss: 3.657 - ETA: 3:09 - loss: 3.657 - ETA: 3:09 - loss: 3.657 - ETA: 3:08 - loss: 3.657 - ETA: 3:08 - loss: 3.657 - ETA: 3:08 - loss: 3.657 - ETA: 3:08 - loss: 3

5652/6000 [===========================>..] - ETA: 2:11 - loss: 3.654 - ETA: 2:11 - loss: 3.654 - ETA: 2:11 - loss: 3.654 - ETA: 2:11 - loss: 3.654 - ETA: 2:10 - loss: 3.653 - ETA: 2:10 - loss: 3.653 - ETA: 2:10 - loss: 3.653 - ETA: 2:10 - loss: 3.653 - ETA: 2:10 - loss: 3.653 - ETA: 2:09 - loss: 3.653 - ETA: 2:09 - loss: 3.653 - ETA: 2:09 - loss: 3.653 - ETA: 2:09 - loss: 3.654 - ETA: 2:09 - loss: 3.653 - ETA: 2:08 - loss: 3.653 - ETA: 2:08 - loss: 3.653 - ETA: 2:08 - loss: 3.653 - ETA: 2:08 - loss: 3.653 - ETA: 2:08 - loss: 3.653 - ETA: 2:07 - loss: 3.653 - ETA: 2:07 - loss: 3.653 - ETA: 2:07 - loss: 3.653 - ETA: 2:07 - loss: 3.653 - ETA: 2:07 - loss: 3.653 - ETA: 2:06 - loss: 3.653 - ETA: 2:06 - loss: 3.652 - ETA: 2:06 - loss: 3.652 - ETA: 2:06 - loss: 3.652 - ETA: 2:06 - loss: 3.652 - ETA: 2:05 - loss: 3.652 - ETA: 2:05 - loss: 3.652 - ETA: 2:05 - loss: 3.652 - ETA: 2:05 - loss: 3.652 - ETA: 2:05 - loss: 3.652 - ETA: 2:04 - loss: 3.653 - ETA: 2:04 - loss: 3.653 - ETA: 2:04 - loss: 3

5988/6000 [============================>.] - ETA: 1:08 - loss: 3.642 - ETA: 1:08 - loss: 3.642 - ETA: 1:08 - loss: 3.642 - ETA: 1:08 - loss: 3.642 - ETA: 1:08 - loss: 3.642 - ETA: 1:07 - loss: 3.642 - ETA: 1:07 - loss: 3.642 - ETA: 1:07 - loss: 3.642 - ETA: 1:07 - loss: 3.642 - ETA: 1:07 - loss: 3.642 - ETA: 1:06 - loss: 3.642 - ETA: 1:06 - loss: 3.642 - ETA: 1:06 - loss: 3.642 - ETA: 1:06 - loss: 3.642 - ETA: 1:06 - loss: 3.642 - ETA: 1:05 - loss: 3.642 - ETA: 1:05 - loss: 3.642 - ETA: 1:05 - loss: 3.642 - ETA: 1:05 - loss: 3.642 - ETA: 1:05 - loss: 3.642 - ETA: 1:04 - loss: 3.642 - ETA: 1:04 - loss: 3.642 - ETA: 1:04 - loss: 3.642 - ETA: 1:04 - loss: 3.642 - ETA: 1:04 - loss: 3.642 - ETA: 1:03 - loss: 3.641 - ETA: 1:03 - loss: 3.642 - ETA: 1:03 - loss: 3.642 - ETA: 1:03 - loss: 3.642 - ETA: 1:03 - loss: 3.642 - ETA: 1:02 - loss: 3.642 - ETA: 1:02 - loss: 3.642 - ETA: 1:02 - loss: 3.642 - ETA: 1:02 - loss: 3.642 - ETA: 1:02 - loss: 3.642 - ETA: 1:01 - loss: 3.642 - ETA: 1:01 - loss: 3

6000/6000 [==============================] - ETA: 2s - loss: 3.638 - ETA: 1s - loss: 3.638 - ETA: 1s - loss: 3.638 - ETA: 1s - loss: 3.638 - ETA: 1s - loss: 3.638 - ETA: 1s - loss: 3.638 - ETA: 0s - loss: 3.638 - ETA: 0s - loss: 3.638 - ETA: 0s - loss: 3.638 - ETA: 0s - loss: 3.637 - ETA: 0s - loss: 3.638 - 1184s 197ms/step - loss: 3.6382
Epoch 1/1
 314/6000 [>.............................] - ETA: 16:57 - loss: 3.97 - ETA: 18:26 - loss: 3.36 - ETA: 19:03 - loss: 3.50 - ETA: 19:03 - loss: 3.52 - ETA: 18:52 - loss: 3.49 - ETA: 18:33 - loss: 3.50 - ETA: 18:29 - loss: 3.32 - ETA: 18:46 - loss: 3.33 - ETA: 18:56 - loss: 3.31 - ETA: 18:45 - loss: 3.31 - ETA: 18:40 - loss: 3.15 - ETA: 18:44 - loss: 3.19 - ETA: 18:49 - loss: 3.21 - ETA: 18:42 - loss: 3.19 - ETA: 18:43 - loss: 3.14 - ETA: 18:44 - loss: 3.07 - ETA: 18:51 - loss: 3.15 - ETA: 18:42 - loss: 3.10 - ETA: 18:31 - loss: 3.12 - ETA: 18:29 - loss: 3.09 - ETA: 18:30 - loss: 3.14 - ETA: 18:29 - loss: 3.09 - ETA: 18:27 - loss: 3.09 - ETA: 1

KeyboardInterrupt: 

In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import argparse
ap = argparse.ArgumentParser()
ap.add_argument('-i', '--image', required=True,help="image path")
args = vars(ap.parse_args())
img_path = args["image"]
def extract_features(filename, model):
        try:
            image = Image.open(filename)
        except:
            print("ERROR: Couldn't open image! Make sure the image path and extension is correct")
        image = image.resize((299,299))
        image = np.array(image)
        # for images that has 4 channels, we convert them into 3 channels
        if image.shape[2] == 4: 
            image = image[..., :3]
        image = np.expand_dims(image, axis=0)
        image = image/127.5
        image = image - 1.0
        feature = model.predict(image)
        return feature
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
return None
def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'start'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        pred = model.predict([photo,sequence], verbose=0)
        pred = np.argmax(pred)
        word = word_for_id(pred, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text
#path = 'Flicker8k_Dataset/111537222_07e56d5a30.jpg'
max_length = 32
tokenizer = load(open("tokenizer.p","rb"))
model = load_model('models/model_1.h5')
xception_model = Xception(include_top=False, pooling="avg")
photo = extract_features(img_path, xception_model)
img = Image.open(img_path)
description = generate_desc(model, tokenizer, photo, max_length)
print("\n\n")
print(description)
plt.imshow(img)

usage: ipykernel_launcher.py [-h] -i IMAGE
ipykernel_launcher.py: error: the following arguments are required: -i/--image


SystemExit: 2

C:\Users\Admin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
